In [1]:
# Dependencies
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine
from census import Census

# Census API Key
from config import census_key
c = Census(census_key, year=2017)

In [ ]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",         
                         ), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)



# Column Reordering and renaming
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)
#Data set snap shot
print(len(census_pd))
census_pd.head()

In [ ]:
# Save data as a csv so we don't have to keep hitting the API
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("./Resources/census_data.csv", encoding="utf-8", index=False)

# Clean data in preparation for insert into Database

In [2]:
#Read in stored data to avoid hit API too often
csv_file = "./Resources/census_data.csv"
census_data_df = pd.read_csv(csv_file, dtype = "str").drop(columns = "Name")
census_data_df.head()

,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,Zipcode,Poverty Rate,Unemployment Rate
0,11757.0,17599.0,38.9,7041.0,11282.0,2454.0,00601,64.105915108813,13.9439740894369
1,16190.0,39209.0,40.9,8978.0,20428.0,2538.0,00602,52.10028309826826,6.473003647121835
2,16645.0,50135.0,40.4,10897.0,25176.0,3588.0,00603,50.21641567767029,7.156676972175127
3,13387.0,6304.0,42.8,5960.0,4092.0,204.0,00606,64.91116751269035,3.236040609137056
4,18741.0,27590.0,41.4,9266.0,12553.0,1474.0,00610,45.498368974266036,5.342515404131932


In [3]:
#Clean the data by dropping duplicates and setting the index
census_data_df.drop_duplicates("Zipcode", inplace=True)
census_data_df.set_index("Zipcode", inplace=True)

census_data_df.head()

,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,Poverty Rate,Unemployment Rate
Zipcode,,,,,,,,
00601,11757.0,17599.0,38.9,7041.0,11282.0,2454.0,64.105915108813,13.9439740894369
00602,16190.0,39209.0,40.9,8978.0,20428.0,2538.0,52.10028309826826,6.473003647121835
00603,16645.0,50135.0,40.4,10897.0,25176.0,3588.0,50.21641567767029,7.156676972175127
00606,13387.0,6304.0,42.8,5960.0,4092.0,204.0,64.91116751269035,3.236040609137056
00610,18741.0,27590.0,41.4,9266.0,12553.0,1474.0,45.498368974266036,5.342515404131932


# Establish database connection & data load

In [5]:
#Setting up connection to postgres local connection
from config import password
connection_string = f"postgres:{password}@localhost:5432/etl_project_db"
engine = create_engine(f'postgresql://{connection_string}')

In [6]:
# Confirm tables
engine.table_names()

[]

In [ ]:
#load data to Censys table in etl_project_db
census_data_df.to_sql(name='census_data', con=engine, if_exists='append', index=True)